In [42]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news/submit.csv
/kaggle/input/fake-news/train.csv
/kaggle/input/fake-news/test.csv


In [43]:
# import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import cProfile
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [44]:
fake_train = pd.read_csv("/kaggle/input/fake-news/train.csv")
fake_test = pd.read_csv("/kaggle/input/fake-news/test.csv")
submit = pd.read_csv("/kaggle/input/fake-news/submit.csv")

In [45]:
print(fake_train.shape)
print(fake_test.shape)

(20800, 5)
(5200, 4)


In [46]:
fake_train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [47]:
fake_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [48]:
fake_train = fake_train.fillna("")
fake_test = fake_test.fillna("")

In [49]:
fake_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20800 non-null  object
 2   author  20800 non-null  object
 3   text    20800 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [50]:
fake_train["label"].value_counts()

label
1    10413
0    10387
Name: count, dtype: int64

# Step 2: Text Preprocessing

nltk.download("stopwords")

In [51]:
fake_train = fake_train.drop("id", axis=1)
fake_test = fake_test.drop("id", axis=1)

fake_train.head(20)

,title,author,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,,Ever wonder how Britain’s most iconic pop pian...,1
7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,Excerpts From a Draft Script for Donald Trump’...,,Donald J. Trump is scheduled to make a highly ...,0
9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


In [52]:
X_train = fake_train.drop("label", axis=1)
y_train = fake_train["label"]
X_test = fake_test.copy()

# Convert feature names to strings
# X_train.columns = X_train.columns.astype(str)
# X_test.columns = X_test.columns.astype(str)

In [53]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   20800 non-null  object
 1   author  20800 non-null  object
 2   text    20800 non-null  object
dtypes: object(3)
memory usage: 487.6+ KB


In [54]:
nltk.download("stopwords")
cachedStopWords = stopwords.words("english")

def clean_text(text):
    # keep only the letters
    text = re.sub("[^a-zA-Z]", " ", text)
    text = text.lower()
    # remove stopwords before tokenization
    text = [word for word in text.split() if word not in cachedStopWords]
    # apply stemming
    ps = PorterStemmer()
    text = [ps.stem(word) for word in text]
    text = ' '.join(text)
    return text
    
X_train = X_train["text"].apply(clean_text)
X_test = X_test["text"].apply(clean_text)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Step 3: Feature Extraction

In [55]:
X_train

0        hous dem aid even see comey letter jason chaff...
1        ever get feel life circl roundabout rather hea...
2        truth might get fire octob tension intellig an...
3        video civilian kill singl us airstrik identifi...
4        print iranian woman sentenc six year prison ir...
                               ...                        
20795    rapper unload black celebr met donald trump el...
20796    green bay packer lost washington redskin week ...
20797    maci today grew union sever great name america...
20798    nato russia hold parallel exercis balkan press...
20799    david swanson author activist journalist radio...
Name: text, Length: 20800, dtype: object

In [56]:
print(X_train.shape)
print(X_test.shape)

(20800,)
(5200,)


In [57]:
print(y_train.shape)

(20800,)


In [58]:
tfidf = TfidfVectorizer(max_features=5000)
X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)

In [59]:
X_train

<20800x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 4337413 stored elements in Compressed Sparse Row format>

In [60]:
print(X_train.shape)
print(X_test.shape)

(20800, 5000)
(5200, 5000)


# Step 5: Model Training

In [65]:
# Decision Tree
dt_classifier = DecisionTreeClassifier()
dt_classifier.fit(X_train, y_train)
# Random Forest
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train, y_train)
# XGBoost
xgb_classifier = XGBClassifier()
xgb_classifier.fit(X_train, y_train)
# Extra Trees
et_classifier = ExtraTreesClassifier()
et_classifier.fit(X_train, y_train)
# LightGBM
lgbm_classifier = LGBMClassifier()
lgbm_classifier.fit(X_train, y_train)

LGBMClassifier()

# Step 6: Evaluation

In [67]:
from sklearn.model_selection import cross_val_score, KFold

kf = KFold(n_splits=5)

dt_results = cross_val_score(dt_classifier, X_train, y_train, scoring="f1", cv=kf)
print(f"\nDecision Tree:\n{dt_results}\n")
rf_results = cross_val_score(rf_classifier, X_train, y_train, scoring="f1", cv=kf)
print(f"\nRandom Forest:\n{rf_results}\n")
xgb_results = cross_val_score(xgb_classifier, X_train, y_train, scoring="f1", cv=kf)
print(f"\nXGBoost:\n{xgb_results}\n")
et_results = cross_val_score(et_classifier, X_train, y_train, scoring="f1", cv=kf)
print(f"\nExtra Trees:\n{et_results}\n")
lgbm_results = cross_val_score(lgbm_classifier, X_train, y_train, scoring="f1", cv=kf)
print(f"\LightGBM:\n{lgbm_results}\n")


Decision Tree:
[0.88280133 0.8651437  0.8793591  0.87367915 0.88449777]


Random Forest:
[0.94337812 0.93353175 0.94308943 0.93913469 0.94044487]


XGBoost:
[0.96215705 0.95298727 0.95799906 0.95316272 0.9565628 ]


Extra Trees:
[0.93581578 0.92750695 0.93574395 0.92801605 0.92922318]

\LightGBM:
[0.96238467 0.95364238 0.96149303 0.95889749 0.96143837]



In [72]:
y_pred = lgbm_classifier.predict(X_test)

In [74]:
y_pred.shape

(5200,)

In [75]:
submit

,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,1
4,20804,1
...,...,...
5195,25995,0
5196,25996,1
5197,25997,0
5198,25998,1


In [80]:
y_true = submit["label"].values
y_true

array([0, 1, 0, ..., 0, 1, 0])

In [81]:
from sklearn.metrics import f1_score

f1_score(y_true, y_pred)

0.6550904770608663